# CITS5014/15 Research Project

### Name: Peter Millitz [23088298]   Date: 17/10/2025

# Deep-Learning-Based Techniques for Ship Object Detection and Classification Using Complex-Valued SAR Imagery

## 1. Introduction

This research seeks to investigate the use of high-resolution complex-valued SLC data products from space-based SAR imagery for the tasks of ship detection, classification, and vessel regression using Deep Learning (DL). The basis for this investigation is the hypothesis that fully leveraging the information in the inherently complex-valued SAR imagery should naturally lead to superior model performance compared to real-valued networks. The underlying motivation is to achieve tangible improvements in the performance of existing SAR ship detection models used for maritime surveillance of IUU fishing activity.

## 2. The SARFish dataset

### 2.1 Overview

SARFish is an imagery dataset for the purpose of training, validating and testing supervised machine learning models on the tasks of ship detection, classification and vessel length regression. SARFish builds on the work of the xView3-SAR dataset by expanding the imagery data to include Single Look Complex (SLC) as well as Ground Range Detected (GRD) imagery data taken directly from the European Space Agency (ESA) Copernicus Programme Open Access Hub Website.  

The pre-processing applied to the Sentinel-1 images to the SARFish dataset was chosen to be deliberately minimal. The only operations applied to the SARFish data have been those necessary to make the images usable for computer vison tasks and include flipping, debursting and no-data masking. The philosophy was to provide GRD and SLC data in a format as close as practicable to the Sentinel-1 data that can be downloaded from Copernicus.  

The three operations applied post download are describe below:  

1. **Flipping:** Flipping was applied to both GRD and SLC products. Due to the acquisition methodology, raw images appear as mirror images of the surface of the Earth. This parity inversion between raw images and real-world coordinates is accounted for by reversing the image and its associated ground control points along the range or x-axis. 

2. **Debursting:** Debursting was applied only to SLC products. Sentinel-1 SLC products are provided as sets of 3 "swaths" per channel per scene. These swaths consist of "sub-swaths" or "bursts" which are overlapping segments of the image. The process of de-bursting is the alignment of these bursts into a contiguous image. This was done to create a one-to-one correspondence between the objects in each swath and the features on the Earth to which they correspond. It is important to note that as the deburst images are concatenations of bursts which themselves are individual SAR images, there are significant phase discontinuities on the boundaries of the bursts. It was decided for the purposes of this dataset that the bursts within the individual swaths should be merged rather than being split into separate images.

3. **No data masking:** No data masking was applied to both GRD and SLC products. Invalid pixels in the image have been masked using a 'no data' mask.

### 2.2. Dataset download

All SLC (and corresponding GRD) products from the 50 scenes that comprise the 'validation' partition of the SARFish dataset, were downloaded for this project. The SARFish dataset resides in a Hugging Face repository available for download at: https://huggingface.co/datasets/ConnorLuckettDSTG/SARFish. The validation partition was chosen because of the high prevalence of associated labelling where true vessel detections were of `HIGH` and `MEDIUM` confidence and bounding box coordinates were present. In contrast, the 'train' partition, although having many more scenes, entirely lacked labels with 'HIGH' confidence true vessel detections. Moreover, the majority of MEDIUM confidence labels did not have bounding boxes. The exact selection criteria for SLC images based on associated groundtruth labels is shown in Table 1 below.

Table 1. Label criteria required for SLC image selection.

|  Attribute |      Value(s)     |
|:-----------|:------------------|
|  is_vessel |        True       |
| is_fishing |   True, False     |
| confidence |   HIGH, MEDIUM    |
|     top    |    not missing    |
|     left   |    not missing    |
|   bottom   |    not missing    |
|    right   |    not missing    |

A script, kindly supplied by one of the principal authors of the dataset^1, was customised to selectively download 50 SLC scenes selected from the validation partition (see script: *downloader/download_specific_files_from_the_SARFish_dataset_mod.py*). The script requires a specific python environment setup. The *requirements.txt* file used to create the python environment is included in the downloader directory.

* Note: due to the debursting operation, each scene comprises three separate swaths (with small overlap), so the total number of images downloaded is actually 150.

[1] Connor Luckett, connor.luckett@defence.cgov.au

## 3. Setup

### 3.1 Import required libraries

In [ ]:
from pathlib import Path
import os
import re
import socket
from time import time

import numpy as np # np.__version__ = 1.26.4 (had to downgrade to version < 2.0)
import pandas as pd
import matplotlib.pyplot as plt # Added by PJM
import yaml
from GeoTiff import load_GeoTiff
from complex_scale_and_norm import process_complex_data
from utilities import *
from batch_sar_processing import *
from compare_crops import crop_compare
from analyse_crop_stats import *

%gui qt
from visualise_labels import scale_sentinel_1_image, SARFish_Plot
#from SARFish_metric import score

pd.set_option('display.max_colwidth', None)

### 3.2 Build root paths for raw and processed data

In [ ]:
# determine the current hostname
hostname = socket.gethostname()
if "kaya" in hostname.lower() or os.getenv("HPC_ENV") == "true":
    system = "kaya"
else:
    system = "local"

In [ ]:
# Load config.yaml file
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

# Build the raw and processed data root paths
sar_data_root = config["data_paths"]["SARFish_root_directory"][system]
gen_data_root = config["data_paths"]["Generated_root_directory"][system]

### 3.3 Split the dataset

First create a full list of qualifying validation partition scenes then partition it into train and test scene lists (90:10).

In [ ]:
# Create a list of the 50 selected scene ids from the'validation' partition of the SARFish dataset downloaded from Hugging Face
command = "awk -F',' '{print $3}' ./SARFish_labels/validation/SLC/SLC_validation_labels.csv | tail -n +2 | sort -u"
fifty_scenes = extract_list_from_command(command, print_summary=True, columns=5, list_name="fifty_scenes")

Ten percent of the 50 scenes will be randomly selected and set aside as the test set for later model evaluation

In [ ]:
# Randomly select 10% of scene_ids from the pool of 50 to be set aside as the test set. 
# Store the remainder as the train set to be used for training the model.
train_scene_ids, test_scene_ids = train_test_split(fifty_scenes , test_ratio=0.1, seed=40)

# Display the list of train_scene_ids and save to CSV file
print(f"Number of train scenes: {len(train_scene_ids)}")
print_list_formatted(train_scene_ids, 5, "train_scene_ids")
save_list_to_txt(train_scene_ids, "train_scene_ids.txt")

# Display the list of test_scene_ids and save to CSV file
print(f"Number of test scenes: {len(test_scene_ids)}")
print_list_formatted(test_scene_ids, 5 , "test_scene_ids")
save_list_to_txt(test_scene_ids, "test_scene_ids.txt")

## 4. Exploratory Data Analysis

### 4.1 Training labels

Load and examine the labels associated with each train scene_id. The labels are sourced from the master SLC 'validation' partition labels CSV file ('*SLC_validation_labels.csv*').

Note: Only images with the appropriate level of labelling will be retained for training.

In [ ]:
# Read the the master SLC 'validation' partition labels CSV file in chunks, filtering on train dataset scene_ids.
df_labels_train = pd.concat(
    filter_rows(chunk, 'scene_id', train_scene_ids)
    for chunk in pd.read_csv('./SARFish_labels/validation/SLC/SLC_validation_labels.csv', chunksize=5000)
)
# Display a few rows
df_labels_train

There are a total of 17,011 label entries across the 45 scenes in the train dataset.

In [ ]:
# Check for duplicates
duplicates_val = df_labels_train[df_labels_train.duplicated(subset=['detect_id'], keep=False)]
duplicates_val['detect_id'].value_counts()

245 duplicate detection IDs were found.

Now filter the the dataframe for labels that meet the requirements:

In [ ]:
# Filter for true vessels detections which are either fishing or not and for which there is HIGH or MEDIUM confidence and
# where the pixels locations of all four corners of the bounding box are not missing
col_range = ['top', 'left', 'bottom', 'right'] # bounding box corner pixels

df_labels_train_filt = df_labels_train[ ( (df_labels_train['is_vessel'] == True) & (df_labels_train['is_fishing'].notnull() ) &  
                                    ( (df_labels_train['confidence'] == 'HIGH') | (df_labels_train['confidence'] == 'MEDIUM') ) & 
                                       df_labels_train[col_range].notnull().all(axis=1)) ]                                   
# Display a few rows
df_labels_train_filt

There are 3,603 labels in the train dataset that meet the selection criteria.

In [ ]:
# Display an abbreviated column version of the previous dataframe
df_labels_train_filt_abv = (df_labels_train_filt[['partition','product_type','scene_id','detect_id','swath_index',
                                                  'detect_scene_column','detect_scene_row','top','left','bottom','right',
                                                  'vessel_length_m','is_vessel','is_fishing','confidence']]
                            .assign(detect_id=lambda x: '...' + x['detect_id'].str[-39:]))
# Display a few rows
df_labels_train_filt_abv.head()

The abbreviated display sample, above, shows the columns used for the selection cirteria: 'is_vessel' == True, 'is_fishing' is not missing and confidence has a valid value and all bounding box coordinates are populated. 

Determine if any any swaths have missing labels:

In [ ]:
# First, get the full set of expected swath indices
expected_swaths = set([1, 2, 3])
# Group by scene_id_ and collect the actual swath indices
swaths_with_labels = df_labels_train_filt.groupby('scene_id')['swath_index'].apply(set).reset_index(name='swaths_with_labels')
# Find the difference between expected and actual for each scene
swaths_with_labels['swaths_missing_labels'] = swaths_with_labels['swaths_with_labels'].apply(lambda x: expected_swaths - x)
# Filter down to scenes missing one or more swaths
scenes_with_gaps = swaths_with_labels[swaths_with_labels['swaths_missing_labels'].apply(len) > 0]
# Display scenes with one or more swaths with missing labels
scenes_with_gaps

Ten scenes have at least one swath without labels.

In [ ]:
# Determine the number of unique scene_id/swath_index combinations present. 
# This will indicate the total number of images that meet the selection criteria.
unique_combinations = df_labels_train_filt[['scene_id', 'swath_index']].drop_duplicates()
count_images = len(unique_combinations)
print(f"Number of unique scene_id/swath_index entries: {count_images}")

In [ ]:
# Count and display the number of 'HIGH' and 'MEDIUM' confidence labels (including duplicates) in the train dataset
labels_count = df_labels_train_filt['confidence'].value_counts()
print(labels_count[['HIGH', 'MEDIUM']])

In [ ]:
# Count and display the number of non-fishing vessel and fishing vessel labels in the train dataset
class_count = df_labels_train_filt[['is_fishing', 'is_vessel']].apply(pd.Series.value_counts)
print(class_count)

Comments:

* There are a total of **17,011** label entries (rows) in total, encompassing 45 scenes. Of these, **245** have duplicated detection ids. This is attributed to the overlap between adjacent swaths i.e., a detection occurring in the overlap zone is recorded once, having a unique detection id, but is associated with both swaths and considered unique to each swath.
* There are a total of **3,603** labels spread across the 45 scenes, which meet the selection criteria. The overwhelming majority are 'HIGH' confidence labels (3559) with only 44 'MEDIUM' confidence labels.
* Of the 3,603 vessel detections, **901** (33.3%) are fishing vessels, indicating a class imbalance of almost exactly **3:1** in favour of non-fishing vessels.
* There are **9** scenes identified that have one swath with no labels that meet the selection criteria plus **1** scene with two swaths with no labels that meet the selecton criteria. Since each swath is equivalent to one unique image product (i.e. '.tiff' file), this means 11 images in the dataset can't be used. This reduces the total number of images in the train dataset to **124**, down from the original 135.
* The bounding box annotations use an **unconventional image origin** (bottom-left) for image coordinates systems. This needs to be accounted for to ensure compatibility downstream of the processing pipeline.

### 4.2 Training images

#### 4.2.1 Statistical summary

Compute summary statistics for each SLC VH-polarisation train scene image.

(Original images are stored as GeoTIFF files and converted to numpy arrays. The arrays are saved to disk for later use.)

In [ ]:
!./compute_sar_stats.py -h

In [ ]:
# Build required paths
input_path  = sar_data_root
arrays_path = config["create_crop"]["arrays_path"]
output_path = os.path.join(gen_data_root, arrays_path)
# Print the input and output paths created
print(f"input_path:  {input_path}\noutput_path: {output_path}")

In [ ]:
# Compute statistics for all train scene SLC VH-polarisation image (.tiff) files
#%run compute_sar_stats.py {input_path} --pattern slc-vh --include-scene-ids train_scene_ids.txt --save-array {output_path}

Read the output file into a dataframe and display the full results:

In [ ]:
# Load the slc-vh stats tabulation  into a dataframe and display it
df_slc_vh_stats = pd.read_csv('slc-vh_stats.csv')
df_slc_vh_stats

Display an abbreviated version of the dataframe showing important columns only:

In [ ]:
# Display abbreviated list of columns
df_display = (df_slc_vh_stats[['scene_id', 'filename', 'valid_pixels', 'nan_count','zero_count', 'real_min', 'real_max',
                               'amplitude_mean','amplitude_std','amplitude_min','amplitude_max','amplitude_median',
                               'phase_circular_mean','phase_min', 'phase_max', 'phase_circular_std']]
              .assign(filename=lambda x: x['filename'].str[:14] + '...'))
df_display

Determine the global minimum and maximum values of amplitude and phase across the 150 images

In [ ]:
cols = ['amplitude_min','amplitude_max', 'real_min', 'real_max', 'phase_min', 'phase_max']
column_maximums = df_display[cols].max()
pd.set_option('display.precision', 10)
print(column_maximums)

#### 4.2.2 Examine one image array

Analyse one full-size image array from the dataset: `scene_id='5c3d986db930f848v'`, `swath_index=2`.

In [ ]:
# Determine the image array filename to examine
row = df_slc_vh_stats[(df_slc_vh_stats['scene_id']=='5c3d986db930f848v') & (df_slc_vh_stats['filename'].str.contains('002_SARFish', case=False, na=False))]
npy_filename = re.sub(r'\.\w+$', '.npy', row['filename'].iloc[0])
# Print the name of the image array file
print(f"Image array filename: {npy_filename}")

In [ ]:
# Load the image array and print some vital stats
slc_data = np.load(f"{output_path}{npy_filename}")
print(f"slc_data size:{slc_data.size}, slc_data.shape: {slc_data.shape}, slc_data dtype:{slc_data.dtype}")

In [ ]:
# Dispaly the computed statistics for that image array
df_display.loc[[row.index[0]]]

Plot histograms of the amplitude and phase diistribution:

In [ ]:
# Extract the raw amplitude and phase data   
#vh_mag = np.abs(slc_data)
#vh_phase = np.angle(slc_data)

In [ ]:
# Plot histograms of raw amplitude and phase (randomly sampled)
sample_size = 10000
vh_mag_sample = np.random.choice(vh_mag.ravel(), sample_size, replace=False)
vh_phase_sample = np.random.choice(vh_phase.ravel(), sample_size, replace=False)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot histogram on the first subplot
axes[0].hist(vh_mag_sample, bins=50, edgecolor='black')
axes[0].set_title("(a) Amplitude")
axes[0].set_xlabel("Amplitude")
axes[0].set_ylabel("Frequency")

# Plot histogram on the second subplot
axes[1].hist(vh_phase_sample, bins=50, edgecolor='black')
axes[1].set_title("(b) Phase")
axes[1].set_xlabel("Phase (radians)")
axes[1].set_ylabel("Frequency")

# Add super title
fig.suptitle("Figure 1. Amplitude and Phase Frequency Distributions (sample size = 10000).", fontsize=16)

# Improve layout and show plot
plt.tight_layout()
plt.show()

# free up memory
vh_mag = None
vh_phase = None
slc_data = None

**Observations:**
* The actual amplitude range for this image is [0.0, 2684.2] and the actual mean/median values are 19.3 and 15.0 respectively. The distribution is right skewed, starts at zero and has a long tail, resembling a Raleigh distribution, which is typical for this type of data (see Figure 1(a)).
* Phase range is centred on zero radians with values in the range $(-\pi, \pi]$ and essentially distribution uniform across the range but with prominent high frequency peaks around $0$, $\pm\pi/2$ and $\pi$ (see Figure 1(b)).
* phase_max is effectively equal to $\pi$ (discounting a small floating-point precision error) while the minimum phase is very close to $-\pi$. These min/max values are characteristic across the 150 images.

**Pre-processing recommendations: Scaling and Normalisation**
* **Amplitude:** Logarithmic (decibel) transformation to compress dynamic range:
  
$\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad vh\_mag\_db = 20\log_{10}(vh\_mag + \epsilon)$,

$\quad~~$ where $\epsilon$ is some small number (e.g. $10^-6$) to prevent log(0) errors, then normalise dB value to [0, 1] using min/max normalisation or percentile clipping to manage outliers.

$\quad~~$ Note: A factor of 20 is appropriate for decibel conversion of field measurements such has amplitude and maintains consistency with radar and SAR processing conventions.

* **Phase:** The focus is on maintaining relative structure. Phase is inherently circular/periodic; representing phase as sine and cosine components respects the circular nature of phase and can be more suitable for training a neural network (eliminates the phase wraparound problem while keeping values in a well-behaved range). This representation only requires mapping the values to within the range expected for YOLO-compatibility ([-1, 1] -> [0,1]):

 $\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad phase\_sin\_norm = \frac{(phase\_sin + 1)}{2}\quad\text{and}$
 
 $\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad phase\_cos\_norm = \frac{(phase\_cos + 1)}{2}$.

#### 4.2.3 Displaying an SLC VH-polarisation image

Plot one swath of an SLC VH-polarisation scene from the validation partition, with 'HIGH' confidence labels.
(The same image array in Section 4.1 is used here again).

Note: The image is scaled, clipped, downsampled and the complex amplitude and phase data mapped into real intensity values for display purposes.

Build the image filepath

In [ ]:
# Read correspondences file
xView3_SLC_GRD_correspondences = pd.read_csv("xView3_SLC_GRD_correspondences.csv") # in current working directory ("SARFish/working/")
# Extract the validation partition row entry for scene_Id '5c3d986db930f848v' from the xView3_SLC_GRD_correspondences.csv file
correspondence = xView3_SLC_GRD_correspondences[xView3_SLC_GRD_correspondences['scene_id'] == '5c3d986db930f848v'].squeeze()
# Print extracted row entry details
correspondence

In [ ]:
# Set the swath index for the selected scene
swath_index = 2 # swath 2 of 3
# Now build the path
measurement_path_SLC = Path(sar_data_root, f"{correspondence['SLC_product_identifier']}.SAFE",
                            "measurement", correspondence[f'SLC_swath_{swath_index}_vh'])
# Check the path by printing it 
measurement_path_SLC

Load, prepare and plot the SLC image

In [ ]:
# Set a scale factor to be used for downsampling the image for display
scale_factor = 2 # retain every other pixel

# Unpack image from GeoTIFF as a NumPy array
data_SLC, nodata_mask_SLC, _, _ = load_GeoTiff(str(measurement_path_SLC))

# Apply decibel scaling to image
scaled_data_SLC = scale_sentinel_1_image(data_SLC, nodata_mask_SLC, product_type = "SLC")
data_SLC = None # free up memory

# Clip the scaled data
clipped_scaled_data_SLC = np.clip(scaled_data_SLC, 15, 60)
scaled_data_SLC = None # free up memory

# Downsample the image
clipped_scaled_data_SLC_resized = clipped_scaled_data_SLC[::scale_factor, ::scale_factor]
clipped_scaled_data_SLC = None # free up memory

# Downsample the nodata mask
nodata_mask_SLC_resized = nodata_mask_SLC[::scale_factor, ::scale_factor]

# Plot the final image
plot_SLC = SARFish_Plot(clipped_scaled_data_SLC_resized, nodata_mask_SLC_resized,
                        title = f"SLC VH-polarisation product with labels, swath: {swath_index}", show=True)

# Clean up remaining intermediate products too free up memory
nodata_mask_SLC_resized = None
clipped_scaled_data_SLC_resized = None

Load and display groundtruth labels

In [ ]:
groundtruth_SLC = pd.read_csv('./SARFish_labels/validation/SLC/SLC_validation_labels.csv')
groundtruth_SLC = groundtruth_SLC[groundtruth_SLC['SLC_product_identifier'] == correspondence['SLC_product_identifier']]
groundtruth_SLC = groundtruth_SLC[groundtruth_SLC['confidence'] == 'HIGH']

In [ ]:
# Scale down the bounding box and target coordinates the match the image scale
groundtruth_SLC[['detect_scene_column', 'detect_scene_row', 'left', 'right', 'bottom', 'top']] = \
groundtruth_SLC[['detect_scene_column', 'detect_scene_row', 'left', 'right', 'bottom', 'top']].apply(lambda x: round(x / scale_factor))

In [ ]:
# Plot the 'HIGH' confidence labels on the image currently displayed in the pop-up interactive window
swath_groundtruth_SLC = groundtruth_SLC[groundtruth_SLC['swath_index'] == swath_index]
plot_SLC.add_bboxes(swath_groundtruth_SLC[['left', 'right', 'bottom', 'top']])
plot_SLC.add_labels(columns = swath_groundtruth_SLC['detect_scene_column'], rows = swath_groundtruth_SLC['detect_scene_row'],
                    categories = swath_groundtruth_SLC[['detect_id', 'is_vessel', 'is_fishing', 'vessel_length_m', 'confidence']],
                    legend_label = "groundtruth", color = "yellow")

In [ ]:
# Capture and display the same image inline using matplotlib
import matplotlib.pyplot as plt

img = plot_SLC.render()  # Capture the VisPy canvas as an image
plt.title("Figure 2. SLC VH-polarisation image with labels")
plt.imshow(img)
plt.axis('off')
plt.show()

plot_SLC = None # free memory

Figure 2 above, is the scaled, clipped and downsampled version of the SLC VH-polarisation product for scene_id '5c3d986db930f848v' (swath 2). The complex amplitude and phase data have been mapped into real intensity values for the display. Yellow boxes represent the bounding-boxes of the groundtruth labels, representing fishing vessels or non-fishing-vessels and where confidence = 'HIGH'.

## 5. Pre-processing

### 5.1 Create image crops

Generate 96 x 96 pixel image crops from the raw image arrays saved to disk in Section 4.1. Each crop should be centred on a positive vessel detection location, with confidence level HIGH or MEDIUM, and have an associated bounding box.

Notes:

* Bounding box (BBox) annotations use inverse y-axis labeling i.e., image origin is bottom-left. This is corrected to top-left on-the-fly in the crop generating script `create_crop.py` below.
* True positive vessel detections with BBoxes that extend beyond crop boundaries by more than 5 pixels are skipped. BBoxes extending by <= 5 pixels are shrunk to match the crop dimensions.
* Crops where detections are close to the original image boundary are zero-padded to maintain a consistent crop-size. Crops with excessive padding (i.e., where padding encroaches on the BBox) are, as a rule, rejected. 
* The crop size of 96 x 96 pixels was chosen because it encapsulates most of the largest vessels in the dataset while minimising the background. Moreover, being a square crop-size of a multiple of 32, it is optimal for more efficient processing by models like YOLO.

In [ ]:
# Create training image crops - see config.yaml for configuration parameters, crops_log for full logging report.
%run create_crop.py --config config.yaml --base-dir {gen_data_root}

In [ ]:
# Print the processing summary from the crops_log
!awk '/PROCESSING SUMMARY/ {found=1} found' ./crops_log

In [ ]:
# Create a list of of all 11 disqualified image arrays for reference
command = "awk -v pat='No crops created' '$0 ~ pat {print fname} {fname = $2}' ./crops_log"
disqualified_images = extract_list_from_command(command, output_file="disqualified_images.txt", print_summary=True, columns=1, list_name="disqualified_images")

Comments:

From Section 4.1, we saw there are a total of 3,603 positive vessel detection labels in the train dataset that fit the criteria defined in Section 2.2. The *crops_log* generated by the `create_crop.py` script reports that of the 135 input swaths (aka images) **3,577** image crops were created. **26** potential crops were skipped because the bounding boxes fell partially outside of the crop boundary by more than 5 pixels while **17** were retained by shrinking the edges of the bounding box by up to 5 pixels at either end. Another **5** crops required padding due to vessel detections very close to the image boundary. On visual inspection, **3** of the padded crops were rejected because the bounding boxes either straddled or lay very close to (within a few pixels) of the padded region. The log also reported that **11** input images failed label validation meaning no qualifying labels were present in the annotations file for these inputs. This was consistent with the observation in Section 4.1, i.e., for the 45 training scenes only 124 out of 135 images (aka swaths) had annotations with 'HIGH' or 'MEDIUM' confidence for positive vessel detections.

### 5.2 Crops analysis

In [ ]:
# Analyse the magntiude and real components of crop pixels (all crops) to determine optimal visualisation parameters
stats, mag_vals, log_vals, real_vals = analyse_crop_statistics('.') # base path to 'images' directory is current working directory

In [ ]:
# Print the formatted statistical summary with recommendations
print_statistics_summary(stats)

In [ ]:
# Plot distribution visualizations
plot_distribution_analysis(stats, mag_vals, log_vals, real_vals)

Comments:

* Only the magnitude and real components, measured across all 3577 crops, are considered here, principally to determine optimal visualisation parameters.
* The recommended normalisaton parameters determined above will be used for visualisations henceforth, where applicable.

### 5.3 Crop visualisation

#### 5.3.1 Interactive visual tool

An interactive tool to visualise the extracted crops was developed. The tool offers a selection of display modes, including magnitude (default) and decibel-scaled magnitude, phase, sin(phase), cos(phase), real and imaginary. The number of randomly selected image samples to view is user-selectable (default=50). It also offers a single-image view mode where the user nominates a specific image to view at launch. Optionally, bounding boxes can also displayed with the class of the detected object indicated ('is_fishing' or 'is_vessel').

In [ ]:
!./visualise_crops.py --help

In [ ]:
# launch the interactive SAR image browser in raw magnitude (default) mode (swichable during the session)
!./visualise_crops.py -i images -l labels

#### 5.3.2 Padded crop QC

Visually examine the 5 padded crops created earlier

In [ ]:
# Create a list of the images that have been padded
command = "awk -v pat='PADDED' '$0 ~ pat {print $2 \".npy\"}' ./crops_log"
padded_images = extract_list_from_command(command, output_file="padded_images.txt", print_summary=True, columns=1, list_name="padded_images")

In [ ]:
# Plot each padded crops in both magnitude and log-magnitude display modes ('real' is an alternative display mode)
crop_compare(padded_images, base_dir='.',
             amp_min=0.0, amp_max=70.0,
             log_min=0.0, log_max=36.9,
             real_min=-35, real_max=35,
             mode_row1='magnitude', mode_row2='log-magnitude')

Comments:

The plots above display the 5 padded crops in two separate display modes, raw magnitude and decibel-scaled magnitude.

The first crop image shows a vessel positioned right up against the padded region border (original image boundary). The vessel signature is well defined, clearly oriented horizontally and consistent with its BBox placement. The faint vertical echoes extending into the padded region are typical of sidelobe artefacts commonly seen in SAR imagery.

In the second crop, the BBox is poorly positioned over a vessel which appears to have a *vertical* rather than the horizontal orientation implied by the BBox. The vessel and BBox are also in very close proximity to the padded region border. It is unclear whether the faint echo emanating directly from the end of the vessel crossing into the padded region, is part of the vessel signature proper or not.

The third crop image displays a highly questionable detection, with the 'vessel' pixels, positioned right on the border of the padded region, and having intensities virtually indistinguishable from the background.

The last two images show two distinct vessel detections, including one of a very large vessel (**100.5 m** in length^2). Despite the close proximity of the larger vessel to the image border, this crop, and the last, are considered acceptable for use in training. The first crop is marginal however the second and third are considered unacceptable. 

[2] Vessel length data obtained from *SLC_validation_labels.csv* sourced from [xView3 website](https://iuu.xview.us/)

### 5.4 Scaling and normalisation

This section applies scaling and percentile clipping to amplitude data and sine and cosine transformations to phase data, in each training image crop. This is followed by normalisation to [0, 1] for YOLO compatiblity. The approach follows the recommendations in Section 4.2.1 and is informed by the statistical analysis of cropped images in Section 5.2. The final processing stage involves stacking the three components (amplitude_norm, phase_sin_norm and phase_cos_norm) in order to simulate the 3-channel RGB image input format expected by YOLO models (3, Height, Width).

Note: With respect to amplitudes, it is assumed that non-fishing vessels display greater variability in amplitude intensity signatures relative to fishing vessels. This is because of their large variability in size, shape and construction. For example, large cargo/container ships generally show very high intensities resulting from corner reflections and metal structures, while smaller passenger ferries display more moderate intensities. Add to this the different orientations and aspect ratios, then a broad range of different backscatter levels could be expected. In contrast, fishing vessel backscatter levels would be expected to be mostly in the moderate range due to smaller size, different construction and materials used (wooden hulls as opposed to metal.

In [ ]:
# Display the core processing script command line parameters and usage examples for standalone use
%run complex_scale_and_norm.py --help

Batch process the image crops created in Section 5.1 using the `batch_process_sar_data() function` which invokes the `complex_scale_and_norm.py` script. 

Use the 1st and 99th percentile of amplitude values (1.0 and 70.0 respectively) for the crops dataset computed in Section 5.2 to clip the amplitudes prior to scaling. 

In [ ]:
%run batch_sar_processing.py --config config.yaml --base-dir {gen_data_root}

In [ ]:
# Print the processing summary from batch_sar_processing.log if needed
#!head -n 5 ./batch_sar_processing.log; tail -n 6 ./batch_sar_processing.log

Load and display vital the stats for one scaled and normalised image crop array created in the previous cell:

In [ ]:
# Build path to the processed crop images
images_proc_path = Path(gen_data_root, config["batch_sar_processing"]["output_dir"])
# Check the path by printing it
print(images_proc_path)

In [ ]:
# Load a single normalised crop image
slc_crop_norm = np.load(f"{images_proc_path}/0d8ed29b0760dc59v_043.21094381000000339554_015.15049506000000079098_swath1_proc.npy")

In [ ]:
# Compute some vital statistics of the normalised crop image
slc_crop_norm_vitals = {
    "size": slc_crop_norm.size, 
    "shape": slc_crop_norm.shape, 
    "dtype": slc_crop_norm.dtype, 
    "amp_min": np.min(slc_crop_norm[:, :, 0]), 
    "amp_max": np.max(slc_crop_norm[:, :, 0]), 
    "phase_sin_min": np.min(slc_crop_norm[:, :, 1]), 
    "phase_sin_max": np.max(slc_crop_norm[:, :, 1]),
    "phase_cos_min": np.min(slc_crop_norm[:, :, 2]), 
    "phase_cos_max": np.max(slc_crop_norm[:, :, 2])    
}
slc_crop_norm_vitals

Shape of array and all channel ranges in expected range (YOLO-compatible)

### 5.5 Data augmentation

[Click here to visit YoTube on Mosaic Data Augmentation](https://www.youtube.com/watch?v=V6uj-eGmE7g&list=PLp5OuA3mFH5JTuKuk07wEMDylDF-Slo2W&index=21)

## 6. Modelling

#### Load the model

In [ ]:
from ultralytics import YOLO
import torch

In [ ]:
# Load the model
model = YOLO('yolo11n.pt')

#### Verify installation

In [ ]:
# Run inference on an image
results = model('https://ultralytics.com/images/bus.jpg')

In [ ]:
# Print detailed results with bounding boxes and confidence
for r in results:
    print(f"Detected {len(r.boxes)} objects")
    
    # Access bounding box information
    if r.boxes is not None:
        boxes = r.boxes.xyxy  # bounding boxes in xyxy format
        confidences = r.boxes.conf  # confidence scores
        classes = r.boxes.cls  # class indices
        
        print("\nDetailed detections:")
        for i, (box, conf, cls_idx) in enumerate(zip(boxes, confidences, classes)):
            x1, y1, x2, y2 = box.tolist()
            class_name = r.names[int(cls_idx)]
            
            print(f"Object {i+1}:")
            print(f"  Class: {class_name} (ID: {int(cls_idx)})")
            print(f"  Confidence: {conf:.4f}")
            print(f"  Bounding box: ({x1:.1f}, {y1:.1f}, {x2:.1f}, {y2:.1f})")
            print(f"  Box format: (x1, y1, x2, y2)")
            print()

In [ ]:
# Save results with bounding boxes drawn
results[0].save('output_with_boxes.jpg')

In [ ]:
# Access the underlying PyTorch model
pytorch_model = model.model

#### Examine the architecture

In [ ]:
# View the model architecture
print(pytorch_model)

UNDER CONSTRUCTION

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, PowerNorm
import gc
from typing import List, Tuple, Dict, Optional

def analyze_vessel_crop(complex_crop, crop_id=None, center_radius=10):
    """
    Analyze a single 64x64 vessel crop to extract vessel and background characteristics.
    
    Parameters:
    -----------
    complex_crop : numpy.ndarray
        64x64 complex-valued SAR crop with vessel at center
    crop_id : str, optional
        Identifier for this crop
    center_radius : int
        Radius around center to consider as vessel region
    
    Returns:
    --------
    dict : Analysis results including vessel/background statistics
    """
    h, w = complex_crop.shape
    center_x, center_y = w // 2, h // 2
    
    # Create masks for vessel (center) and background regions
    y_coords, x_coords = np.ogrid[:h, :w]
    vessel_mask = ((x_coords - center_x)**2 + (y_coords - center_y)**2) <= center_radius**2
    background_mask = ~vessel_mask
    
    # Extract amplitude and phase
    amplitude = np.abs(complex_crop)
    phase = np.angle(complex_crop)
    
    # Vessel statistics
    vessel_amp = amplitude[vessel_mask]
    vessel_phase = phase[vessel_mask]
    
    # Background statistics  
    bg_amp = amplitude[background_mask]
    bg_phase = phase[background_mask]
    
    # Remove invalid values
    vessel_amp = vessel_amp[np.isfinite(vessel_amp) & (vessel_amp > 0)]
    vessel_phase = vessel_phase[np.isfinite(vessel_phase)]
    bg_amp = bg_amp[np.isfinite(bg_amp) & (bg_amp > 0)]
    bg_phase = bg_phase[np.isfinite(bg_phase)]
    
    return {
        'crop_id': crop_id,
        'vessel_amp_mean': np.mean(vessel_amp) if len(vessel_amp) > 0 else 0,
        'vessel_amp_std': np.std(vessel_amp) if len(vessel_amp) > 0 else 0,
        'vessel_amp_max': np.max(vessel_amp) if len(vessel_amp) > 0 else 0,
        'bg_amp_mean': np.mean(bg_amp) if len(bg_amp) > 0 else 0,
        'bg_amp_std': np.std(bg_amp) if len(bg_amp) > 0 else 0,
        'contrast_ratio': (np.mean(vessel_amp) / np.mean(bg_amp)) if len(bg_amp) > 0 and np.mean(bg_amp) > 0 else 0,
        'vessel_pixels': len(vessel_amp),
        'bg_pixels': len(bg_amp),
        'vessel_phase_var': np.var(vessel_phase) if len(vessel_phase) > 0 else 0,
        'bg_phase_var': np.var(bg_phase) if len(bg_phase) > 0 else 0
    }

def create_crop_histogram(complex_crops, crop_ids=None, n_bins_phase=64, n_bins_amp=32,
                         analysis_type='vessel_focused', center_radius=10):
    """
    Create phase-amplitude histogram for vessel crops.
    
    Parameters:
    -----------
    complex_crops : list or numpy.ndarray
        List of 64x64 complex crops, or single 3D array (N, 64, 64)
    crop_ids : list, optional
        Identifiers for each crop
    n_bins_phase : int
        Number of phase bins
    n_bins_amp : int  
        Number of amplitude bins
    analysis_type : str
        'vessel_focused' - Focus on vessel regions only
        'full_crop' - Analyze entire crop
        'comparative' - Compare vessel vs background
    center_radius : int
        Radius for vessel region definition
    """
    
    # Handle input format
    if isinstance(complex_crops, np.ndarray):
        if complex_crops.ndim == 2:
            complex_crops = [complex_crops]
        elif complex_crops.ndim == 3:
            complex_crops = [complex_crops[i] for i in range(complex_crops.shape[0])]
    
    n_crops = len(complex_crops)
    if crop_ids is None:
        crop_ids = [f"Crop_{i:03d}" for i in range(n_crops)]
    
    print(f"Processing {n_crops} vessel crops...")
    
    # Collect statistics for all crops
    crop_stats = []
    all_vessel_amp = []
    all_vessel_phase = []
    all_bg_amp = []
    all_bg_phase = []
    
    for i, crop in enumerate(complex_crops):
        if crop.shape != (64, 64):
            print(f"Warning: Crop {i} has shape {crop.shape}, expected (64, 64)")
            continue
            
        stats = analyze_vessel_crop(crop, crop_ids[i], center_radius)
        crop_stats.append(stats)
        
        # Extract vessel and background data
        h, w = crop.shape
        center_x, center_y = w // 2, h // 2
        y_coords, x_coords = np.ogrid[:h, :w]
        vessel_mask = ((x_coords - center_x)**2 + (y_coords - center_y)**2) <= center_radius**2
        
        amplitude = np.abs(crop)
        phase = np.angle(crop)
        
        if analysis_type == 'vessel_focused':
            vessel_amp_crop = amplitude[vessel_mask]
            vessel_phase_crop = phase[vessel_mask]
            valid_mask = np.isfinite(vessel_amp_crop) & (vessel_amp_crop > 0) & np.isfinite(vessel_phase_crop)
            all_vessel_amp.extend(vessel_amp_crop[valid_mask])
            all_vessel_phase.extend(vessel_phase_crop[valid_mask])
            
        elif analysis_type == 'full_crop':
            amp_flat = amplitude.ravel()
            phase_flat = phase.ravel()
            valid_mask = np.isfinite(amp_flat) & (amp_flat > 0) & np.isfinite(phase_flat)
            all_vessel_amp.extend(amp_flat[valid_mask])
            all_vessel_phase.extend(phase_flat[valid_mask])
            
        else:  # comparative
            # Vessel region
            vessel_amp_crop = amplitude[vessel_mask]
            vessel_phase_crop = phase[vessel_mask]
            valid_vessel = np.isfinite(vessel_amp_crop) & (vessel_amp_crop > 0) & np.isfinite(vessel_phase_crop)
            all_vessel_amp.extend(vessel_amp_crop[valid_vessel])
            all_vessel_phase.extend(vessel_phase_crop[valid_vessel])
            
            # Background region  
            bg_amp_crop = amplitude[~vessel_mask]
            bg_phase_crop = phase[~vessel_mask]
            valid_bg = np.isfinite(bg_amp_crop) & (bg_amp_crop > 0) & np.isfinite(bg_phase_crop)
            all_bg_amp.extend(bg_amp_crop[valid_bg])
            all_bg_phase.extend(bg_phase_crop[valid_bg])
    
    # Convert to numpy arrays
    all_vessel_amp = np.array(all_vessel_amp)
    all_vessel_phase = np.array(all_vessel_phase)
    
    if len(all_vessel_amp) == 0:
        print("Error: No valid vessel pixels found!")
        return None, None, None
    
    print(f"Collected {len(all_vessel_amp):,} vessel pixels from {n_crops} crops")
    if analysis_type == 'comparative':
        print(f"Collected {len(all_bg_amp):,} background pixels")
        print(f"Vessel amplitude range: {np.min(all_vessel_amp):.4f} - {np.max(all_vessel_amp):.4f}")
        if len(all_bg_amp) > 0:
            print(f"Background amplitude range: {np.min(all_bg_amp):.4f} - {np.max(all_bg_amp):.4f}")
    
    # Phase bins
    phase_edges = np.linspace(-np.pi, np.pi, n_bins_phase + 1)
    
    # Create histograms
    if analysis_type == 'comparative' and len(all_bg_amp) > 0:
        all_bg_amp = np.array(all_bg_amp)
        all_bg_phase = np.array(all_bg_phase)
        
        # For comparative analysis, create amplitude bins that cover both vessel and background
        combined_amp = np.concatenate([all_vessel_amp, all_bg_amp])
        amp_min = np.percentile(combined_amp, 1)
        amp_max = np.percentile(combined_amp, 99.5)
        
        # Ensure minimum value is positive for log scale
        amp_min = max(amp_min, 1e-6)
        amp_max = max(amp_max, amp_min * 10)  # Ensure meaningful range
        
        print(f"Combined amplitude range for binning: {amp_min:.6f} - {amp_max:.6f}")
        
        try:
            amp_edges = np.logspace(np.log10(amp_min), np.log10(amp_max), n_bins_amp + 1)
        except ValueError as e:
            print(f"Error creating log bins: {e}")
            print("Falling back to linear binning")
            amp_edges = np.linspace(amp_min, amp_max, n_bins_amp + 1)
        
        # Create separate histograms for vessel and background
        hist_vessel, _, _ = np.histogram2d(all_vessel_phase, all_vessel_amp, 
                                         bins=[phase_edges, amp_edges])
        hist_bg, _, _ = np.histogram2d(all_bg_phase, all_bg_amp,
                                     bins=[phase_edges, amp_edges])
    else:
        # Create amplitude bins - adaptive to vessel data range only
        amp_min = np.percentile(all_vessel_amp, 1)
        amp_max = np.percentile(all_vessel_amp, 99.5)
        
        # Ensure minimum value is positive for log scale
        amp_min = max(amp_min, 1e-6)
        amp_max = max(amp_max, amp_min * 10)  # Ensure meaningful range
        
        try:
            amp_edges = np.logspace(np.log10(amp_min), np.log10(amp_max), n_bins_amp + 1)
        except ValueError as e:
            print(f"Error creating log bins: {e}")
            print("Falling back to linear binning")
            amp_edges = np.linspace(amp_min, amp_max, n_bins_amp + 1)
        
        # Create visualization
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
        
        # Vessel histogram
        im1 = ax1.imshow(hist_vessel.T + 1, extent=[-np.pi, np.pi, 0, n_bins_amp],
                        origin='lower', aspect='auto', cmap='hot', norm=LogNorm())
        ax1.set_title(f'Vessel Regions (n={n_crops} crops)')
        ax1.set_xlabel('Phase (radians)')
        ax1.set_ylabel('Amplitude Bin')
        plt.colorbar(im1, ax=ax1, label='Frequency')
        
        # Background histogram
        im2 = ax2.imshow(hist_bg.T + 1, extent=[-np.pi, np.pi, 0, n_bins_amp],
                        origin='lower', aspect='auto', cmap='blues', norm=LogNorm())
        ax2.set_title('Background Regions')
        ax2.set_xlabel('Phase (radians)')
        ax2.set_ylabel('Amplitude Bin')
        plt.colorbar(im2, ax=ax2, label='Frequency')
        
        # Phase comparison
        phase_centers = (phase_edges[:-1] + phase_edges[1:]) / 2
        vessel_phase_dist = np.sum(hist_vessel, axis=1)
        bg_phase_dist = np.sum(hist_bg, axis=1)
        
        ax3.plot(phase_centers, vessel_phase_dist, 'r-', label='Vessel', linewidth=2)
        ax3.plot(phase_centers, bg_phase_dist, 'b-', label='Background', linewidth=2)
        ax3.set_xlabel('Phase (radians)')
        ax3.set_ylabel('Frequency')
        ax3.set_title('Phase Distribution Comparison')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        
        # Amplitude comparison
        amp_centers = np.sqrt(amp_edges[:-1] * amp_edges[1:])
        vessel_amp_dist = np.sum(hist_vessel, axis=0)
        bg_amp_dist = np.sum(hist_bg, axis=0)
        
        ax4.loglog(amp_centers, vessel_amp_dist + 1, 'r-', label='Vessel', linewidth=2)
        ax4.loglog(amp_centers, bg_amp_dist + 1, 'b-', label='Background', linewidth=2)
        ax4.set_xlabel('Amplitude')
        ax4.set_ylabel('Frequency')
        ax4.set_title('Amplitude Distribution Comparison')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
        
        main_hist = hist_vessel
        
    else:
        # Single histogram
        hist_2d, _, _ = np.histogram2d(all_vessel_phase, all_vessel_amp,
                                     bins=[phase_edges, amp_edges])
        
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
        
        # Main histogram
        im1 = ax1.imshow(hist_2d.T + 1, extent=[-np.pi, np.pi, 0, n_bins_amp],
                        origin='lower', aspect='auto', cmap='plasma', norm=PowerNorm(gamma=0.5))
        title = f'Vessel Crops Phase-Amplitude ({analysis_type})'
        ax1.set_title(title)
        ax1.set_xlabel('Phase (radians)')
        ax1.set_ylabel('Amplitude Bin')
        plt.colorbar(im1, ax=ax1, label='Frequency')
        
        # Phase distribution
        phase_centers = (phase_edges[:-1] + phase_edges[1:]) / 2
        phase_dist = np.sum(hist_2d, axis=1)
        ax2.plot(phase_centers, phase_dist, 'g-', linewidth=2)
        ax2.set_xlabel('Phase (radians)')
        ax2.set_ylabel('Frequency')
        ax2.set_title('Phase Distribution')
        ax2.grid(True, alpha=0.3)
        
        # Amplitude distribution
        amp_centers = np.sqrt(amp_edges[:-1] * amp_edges[1:])
        amp_dist = np.sum(hist_2d, axis=0)
        ax3.loglog(amp_centers, amp_dist + 1, 'purple', linewidth=2)
        ax3.set_xlabel('Amplitude')
        ax3.set_ylabel('Frequency')
        ax3.set_title('Amplitude Distribution')
        ax3.grid(True, alpha=0.3)
        
        # Crop statistics summary
        if crop_stats:
            contrast_ratios = [s['contrast_ratio'] for s in crop_stats if s['contrast_ratio'] > 0]
            vessel_amps = [s['vessel_amp_mean'] for s in crop_stats if s['vessel_amp_mean'] > 0]
            
            ax4.hist(contrast_ratios, bins=20, alpha=0.7, color='orange', edgecolor='black')
            ax4.set_xlabel('Vessel/Background Contrast Ratio')
            ax4.set_ylabel('Number of Crops')
            ax4.set_title(f'Contrast Distribution (n={len(contrast_ratios)})')
            ax4.grid(True, alpha=0.3)
        
        main_hist = hist_2d
    
    # Format phase ticks
    phase_ticks = [-np.pi, -np.pi/2, 0, np.pi/2, np.pi]
    phase_labels = ['-π', '-π/2', '0', 'π/2', 'π']
    for ax in fig.get_axes():
        if 'Phase' in ax.get_xlabel():
            ax.set_xticks(phase_ticks)
            ax.set_xticklabels(phase_labels)
    
    plt.tight_layout()
    
    # Summary statistics
    summary = {
        'n_crops': n_crops,
        'total_pixels': len(all_vessel_amp),
        'amp_range': (np.min(all_vessel_amp), np.max(all_vessel_amp)),
        'mean_contrast': np.mean([s['contrast_ratio'] for s in crop_stats if s['contrast_ratio'] > 0]),
        'crop_stats': crop_stats
    }
    
    return fig, main_hist, summary

def batch_process_crops(crop_directory=None, complex_crops=None, **kwargs):
    """
    Batch process multiple vessel crops.
    
    Parameters:
    -----------
    crop_directory : str, optional
        Directory containing .npy files of crops
    complex_crops : list/array, optional  
        Direct array of crops
    **kwargs : additional arguments for create_crop_histogram
    """
    
    if crop_directory is not None:
        import os
        crop_files = [f for f in os.listdir(crop_directory) if f.endswith('.npy')]
        crop_ids = [f.replace('.npy', '') for f in crop_files]
        complex_crops = [np.load(os.path.join(crop_directory, f)) for f in crop_files]
        print(f"Loaded {len(complex_crops)} crops from {crop_directory}")
    
    elif complex_crops is None:
        print("Error: Must provide either crop_directory or complex_crops")
        return None
        
    return create_crop_histogram(complex_crops, crop_ids, **kwargs)

# Example usage
if __name__ == "__main__":
    # Example: Create synthetic vessel crops for testing
    print("Creating example vessel crops...")
    
    # Generate synthetic 64x64 crops with vessels at center
    n_test_crops = 5
    test_crops = []
    
    for i in range(n_test_crops):
        # Create background sea clutter
        crop = (np.random.randn(64, 64) + 1j * np.random.randn(64, 64)) * 0.5
        
        # Add vessel signature at center
        center_x, center_y = 32, 32
        vessel_size = np.random.randint(6, 12)
        vessel_amp = np.random.uniform(3, 8)
        
        y_coords, x_coords = np.ogrid[:64, :64]
        vessel_mask = ((x_coords - center_x)**2 + (y_coords - center_y)**2) <= vessel_size**2
        
        # Add vessel with higher amplitude and specific phase pattern
        vessel_phase = np.random.uniform(-np.pi, np.pi)
        crop[vessel_mask] = vessel_amp * np.exp(1j * vessel_phase)
        
        test_crops.append(crop)
    
    # Test different analysis types
    analysis_types = ['vessel_focused', 'full_crop', 'comparative']
    
    for analysis in analysis_types:
        print(f"\n{'='*50}")
        print(f"Analysis type: {analysis}")
        print('='*50)
        
        fig, hist, summary = create_crop_histogram(
            test_crops,
            analysis_type=analysis,
            n_bins_phase=32,
            n_bins_amp=24,
            center_radius=8
        )
        
        if summary:
            print(f"Processed {summary['n_crops']} crops")
            print(f"Total pixels analyzed: {summary['total_pixels']:,}")
            print(f"Amplitude range: {summary['amp_range'][0]:.2f} - {summary['amp_range'][1]:.2f}")
            print(f"Mean contrast ratio: {summary['mean_contrast']:.2f}")
        
        plt.show()
        
        cont = input("Continue to next analysis? (y/n): ")
        if not cont.lower().startswith('y'):
            break